In [7]:
!pip install Pillow opencv-python numpy tensorflow

   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 4.7/385.0 MB 25.9 MB/s eta 0:00:15
   - -------------------------------------- 9.7/385.0 MB 24.1 MB/s eta 0:00:16
   - -------------------------------------- 13.6/385.0 MB 23.7 MB/s eta 0:00:16
   - -------------------------------------- 17.0/385.0 MB 21.1 MB/s eta 0:00:18
   -- ------------------------------------- 20.4/385.0 MB 19.9 MB/s eta 0:00:19
   -- ------------------------------------- 23.9/385.0 MB 19.1 MB/s eta 0:00:19
   -- ------------------------------------- 27.3/385.0 MB 18.8 MB/s eta 0:00:20
   --- ------------------------------------ 30.9/385.0 MB 18.7 MB/s eta 0:00:19
   --- ------------------------------------ 36.2/385.0 MB 19.5 MB/s eta 0:00:18
   ---- ----------------------------------- 41.2/385.0 MB 20.0 MB/s eta 0:00:18
   ---- ----------------------------------- 45.6/385.0 MB 20.2 MB/s eta 0:00:17
   ----- ---------------------------------- 50.1/38

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\markdown_py.exe' -> 'C:\\Python311\\Scripts\\markdown_py.exe.deleteme'



# Crop and resize images

The images are 3552x3552 pixels and contain a lot of empty space at the edges. Here we crop the images to 2048x2048 toward the center. Then they are resized to 64x64.

In [30]:
from PIL import Image
import os
from pprint import pprint

dirs = [ (f'.\\data\\raw_data\\{num}', f'.\\data\\processed_64\\{num}') for num in range(16) ]
# dirs = [("./data/sanity_check_raw", "./data/sanity_check")]

pprint(dirs)
for (in_dir, out_dir) in dirs:
    os.makedirs(out_dir, exist_ok=True)

original_size = 3552
crop_size = 2048
target_size = 64
max_files = 60 # We want the same number of images of each ball

[('./data/sanity_check_raw', './data/sanity_check')]


In [40]:
left = (original_size - crop_size) // 2
top = (original_size - crop_size) // 2
right = (original_size + crop_size) // 2
bottom = (original_size + crop_size) // 2

# Values for phone pictures
# top = 200
# left = 360
# bottom = 760
# right = 820

for (in_dir, out_dir) in dirs:
    files = os.listdir(in_dir)
    print(f"Processing {in_dir} ...")

    counter = 0
    for filename in files[:max_files]:
        if filename.lower().endswith(".jpg"):
            counter += 1
            # Open the image
            img_path = os.path.join(in_dir, filename)
            output_path = os.path.join(out_dir, filename)
            
            Image.open(img_path
                ).crop((left, top, right, bottom)
                ).resize((target_size, target_size), Image.Resampling.LANCZOS
                ).save(output_path)

            # print(f"Cropped and saved: {output_path} ({counter} of {max_files})")

Processing ./data/sanity_check_raw ...


# Take a test split randomly from the data

In [3]:
import pathlib
import random
import shutil
import os

random.seed(0)
data_dir = pathlib.Path('./data/processed_64').with_suffix('')
# Access the data like so: 
#   ball8 = list(data_dir.glob(('8/*.jpg')))
#   Image.open(str(ball8[0]))

# Take a test data split
test_items_per_category = 10

for dir in os.listdir('./data/processed_64'):
    image_paths = list(data_dir.glob(f'{dir}/*.jpg'))
    test_split = random.sample(image_paths, test_items_per_category)
    rest_split = [ item for item in image_paths if item not in test_split]
    
    test_dir = f'./data/test/{dir}'
    rest_dir = f'./data/train_and_validation/{dir}'
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(rest_dir, exist_ok=True)

    for path in test_split:
        shutil.move(path, test_dir)
    for path in rest_split:
        shutil.move(path, rest_dir)

shutil.rmtree('./data/processed_64')

In [13]:
import keras

batch_size = 25
img_width = 64
img_height = 64

data_dir = pathlib.Path('./data/train_and_validation').with_suffix('')
test_dir = pathlib.Path('./data/test').with_suffix('')

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="both",
    seed=1,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
test_ds = keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width)
)

class_names = train_ds.class_names
print(class_names)

Found 800 files belonging to 16 classes.
Using 720 files for training.
Using 80 files for validation.
Found 160 files belonging to 16 classes.
['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9']


# Data augmentation

In [5]:
from keras import layers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

normalization_layer = keras.layers.Rescaling(1./255) # Rescale RGB values from 0..255 to floats in 0..1

data_aug = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomBrightness(0.1)
])

batch_size = 30
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
    if shuffle:
        ds = ds.shuffle(1000)
    
    if augment:
        ds = ds.map(lambda x, y: (data_aug(x, training=True), y), num_parallel_calls=AUTOTUNE)
    
    return ds.prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

# Create and train the CNN

In [6]:
num_classes = 16
model = keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer="adam", loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

In [7]:
epochs = 30
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs = epochs
)

Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.0997 - loss: 15.4631 - val_accuracy: 0.2500 - val_loss: 2.5651
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.2324 - loss: 2.3965 - val_accuracy: 0.5250 - val_loss: 1.1906
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5362 - loss: 1.3623 - val_accuracy: 0.5875 - val_loss: 1.2517
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6787 - loss: 0.9481 - val_accuracy: 0.8250 - val_loss: 0.5721
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7029 - loss: 0.8156 - val_accuracy: 0.7000 - val_loss: 1.0519
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7258 - loss: 0.7319 - val_accuracy: 0.8125 - val_loss: 0.4855
Epoch 7/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8294 - loss: 0.5036 - val_accuracy: 0.8125 - val_loss: 0.4658
Epoch 8/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8270 - loss: 0.4578 - val_accuracy: 0.8125 - 

In [8]:
loss, acc = model.evaluate(test_ds)
print("Loss: ", loss, "\n", "Accuracy: ", acc)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9476 - loss: 0.1727
Loss:  0.11524226516485214 
 Accuracy:  0.9624999761581421


In [48]:
image = keras.utils.load_img(".\\data\\sanity_check\\14.JPG")

image_array = keras.utils.img_to_array(image)
image_array = tf.expand_dims(image_array, 0) # Create a batch

predictions = model.predict(image_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
This image most likely belongs to 14 with a 91.89 percent confidence.
